In [1]:
import sys

# append root project directory so packages will be available
sys.path.append('../..')

In [2]:
import einops
import numpy as np
from tqdm import tqdm
from pathlib import Path


import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import DataLoader
from torchvision import transforms

import src.utility.symbols_loader as sl
from src.architectures.course_autoencoder import Autoencoder, Encoder, Decoder
from src.architectures.letters_dataset import LettersDataset

In [3]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
torch.backends.cudnn.benchmark = True # should improve speed if input size don't change

In [4]:
LATENT_WIDTH = 16
BATCH_SIZE = 1024
EPOCHS = 500
LEARNING_RATE = 1e-2
NUM_WORKERS = 2

FILE_PATH = Path().parent.as_posix() + "/" + "../../data/models/"

In [5]:
trials = ['trial_0', 'trial_1', 'trial_2', 'trial_3', 'trial_4']

for trial in trials:
    mnist_dataset = LettersDataset(sl.load_emnist_pages(trial=trial))

    emnist_loader = DataLoader(
    mnist_dataset,
    batch_size=BATCH_SIZE,
    num_workers=NUM_WORKERS,  # 4 * GPU_COUNT
    shuffle=True,
    pin_memory=True
    )
    emnist_Encoder = Encoder(LATENT_WIDTH)
    emnist_Decoder = Decoder(emnist_Encoder.latent_width)
    emnist_Autoencoder = Autoencoder(emnist_Encoder, emnist_Decoder)

    loss_func = F.mse_loss
    optimizer = torch.optim.Adam(emnist_Autoencoder.parameters(), lr=LEARNING_RATE) #, lr=LEARNING_RATE)

    emnist_Autoencoder.to(device)

    for epoch in (pbar := tqdm(range(EPOCHS))):

        emnist_Autoencoder.train()

        for batch in emnist_loader:
                    images = batch.to(device)
                    optimizer.zero_grad()  # MUST be called on every batch
                    predictions, latents = emnist_Autoencoder(images)
                    loss = loss_func(predictions, images)
                    loss.backward()
                    optimizer.step()
        pbar.set_postfix_str("Loss: " + str(loss.item()))

    name = trial + "_emnist_autoencoder.pth"
    torch.save(emnist_Autoencoder.state_dict(), FILE_PATH + name)
    print(f'{trial} done')

100%|██████████| 250/250 [22:03<00:00,  5.29s/it, Loss: 0.041568852961063385]


----------------------------------------------------------------------------------------------------------------------------------------------------------------------------

# KUZUSHIJI

In [14]:
trials = ['trial_0', 'trial_1', 'trial_2', 'trial_3', 'trial_4']

for trial in trials:
    kuzushiji_dataset = LettersDataset(sl.load_kuzushiji_pages(trial=trial))
    kuzushiji_loader = DataLoader(
    kuzushiji_dataset,
    batch_size=BATCH_SIZE,
    num_workers=NUM_WORKERS,  # 4 * GPU_COUNT
    shuffle=True,
    pin_memory=True
    )
    kuzushiji_Encoder = Encoder(LATENT_WIDTH)
    kuzushiji_Decoder = Decoder(kuzushiji_Encoder.latent_width)
    kuzushiji_Autoencoder = Autoencoder(kuzushiji_Encoder, kuzushiji_Decoder)

    loss_func = F.mse_loss
    optimizer = torch.optim.Adam(kuzushiji_Autoencoder.parameters(), lr=LEARNING_RATE) #, lr=LEARNING_RATE)

    kuzushiji_Autoencoder.to(device)
    for epoch in (pbar := tqdm(range(EPOCHS))):

        kuzushiji_Autoencoder.train()

        for batch in kuzushiji_loader:
                    images = batch.to(device)
                    optimizer.zero_grad()  # MUST be called on every batch
                    predictions, latents = kuzushiji_Autoencoder(images)
                    loss = loss_func(predictions, images)
                    loss.backward()
                    optimizer.step()
        pbar.set_postfix_str("Loss: " + str(loss.item()))

    name = trial + "_kuzushiji_autoencoder.pth"
    torch.save(kuzushiji_Autoencoder.state_dict(), FILE_PATH + name)
    print(f'{trial} done')